In [ ]:
df

In [ ]:
cols = df.columns[2:13]
df_selec_staring = df[['gu_name', 'dong_name']]
warnings.filterwarnings(action='ignore')
for col in cols:
    df_selec_staring[col] = pd.qcut(df[col], 10, labels=False, duplicates = 'drop')
warnings.filterwarnings(action='default')

In [ ]:
df_selec_staring['total_star'] = df_selec_staring.iloc[:, 2:].sum(axis=1)
df_selec_staring

In [ ]:
df_selec_staring.groupby(['gu_name', 'dong_name'])

# 지역추천 알고리즘 - Simple Aggregate

__필요한 라이브러리__

In [1]:
import pandas as pd
import numpy as np
import warnings

import psycopg2   
import sql
from sqlalchemy import create_engine

In [ ]:
user = 'postgres' # postgresql DB user ID
password = '7089' # password
host_product = '192.168.0.27' # 서버 IP 
dbname = 'seoul' # DB Name
port='5432' # port number

product_connection_string = "dbname={dbname} user={user} host={host} password={password} port={port}".format(dbname=dbname,
                                    user=user,
                                    host=host_product,
                                    password=password,
                                    port=port) 

In [ ]:
def gu_uj_df_from_db(gu_selec, uj_selec):
    try:
        conn = psycopg2.connect(product_connection_string)
    except:
        print("I am unable to connect to the database")

    cur = conn.cursor() 
    # 여기까지 DB접속을 위해서 항상 필요 
    
    gu_uj_query = ""
    # DB의 table을 DataFrame으로 가져오기
    cur.execute("SELECT * FROM mpop.move_pop;") # table_nm 테이블 select 

#     table_nm = pd.DataFrame(cur.fetchall())
#     table_nm.columns = [desc[0] for desc in cur.description] # 컬럼명 가져오고 싶을때 사용 
    return pd.DataFrame(cur.fetchall())

## aggregate된 database view table 불러오기

In [2]:
df = pd.read_csv('star_table.csv')

## 검색조건 적용 후, scoring

> __함수명__ : star_list_poc  
> __파라미터__ : df 
> > aggregate된 DataFrame

In [3]:
def star_list_poc(df):
    # 필요 리스트 설정
    upjong_cols = df.columns[13:].to_list()
    agg_cols = df.columns[2:13].to_list()
    gu_list = list(df.gu_name.unique())
    
    # 입력받기
    print("✔ 구를 선택하시오")
    [print(u, end=' | ') for u in gu_list]
    print()
    gu_selec = input()

    print()
    print("✔ 업종을 선택하시오")
    [print(u, end=' | ') for u in upjong_cols]
    print()
    uj_selec = input()
    
    print()
    print('📊 %s의'%gu_selec, uj_selec, '업종 추천지역 Top10 결과 📊')
    # 선택된 테이블
    df_selec = df[df.gu_name == gu_selec].loc[:, ['gu_name', 'dong_name', 
                                                  '인구이동총합', '지하철이용객총합',
                                                  '지하철개수', '버스이용객총합', 
                                                  '버스정류장개수', '대학상권개수', 
                                                  '공원시설개수', '집객시설개수', 
                                                  '문화시설개수', '주차장개수', 
                                                  '주차장면적', uj_selec]]

    # qcut 적용할 컬럼
    cols = df_selec.columns.difference(['gu_name', 'dong_name'])
    # qcut 적용 테이블
    df_selec_staring = df_selec[['gu_name', 'dong_name']]
    # 오류 무시 setting
    warnings.filterwarnings(action='ignore')
    # 컬럼별 qcut 적용
    for col in cols:
        df_selec_staring[col] = pd.qcut(df_selec[col], 10, labels=False, 
                                        duplicates = 'drop')
    # 오류 무시 unset
    warnings.filterwarnings(action='default')

    # 총합
    df_selec_staring['total_star'] = df_selec_staring.iloc[:, 2:].sum(axis=1)
    
    # 정렬
    df_star_final = df_selec_staring.sort_values(by='total_star', 
                                                 ascending=False)[['gu_name', 'dong_name', 'total_star']]
#     df_star_final['total_star'] = pd.qcut(df_star_final.total_star, 11, labels=False)
    
    return df_star_final.head(10)

## scoring 실행하기

In [4]:
star_list_poc(df)

✔ 구를 선택하시오
광진구 | 송파구 | 강남구 | 관악구 | 마포구 | 도봉구 | 성북구 | 금천구 | 은평구 | 용산구 | 강북구 | 동대문구 | 구로구 | 중구 | 강동구 | 중랑구 | 종로구 | 노원구 | 성동구 | 양천구 | 서대문구 | 강서구 | 서초구 | 동작구 | 영등포구 | 
강남구

✔ 업종을 선택하시오
분식전문점 | 호프.간이주점 | 양식음식점 | 제과점 | 한식음식점 | 중식음식점 | 편의점 | 일식음식점 | 커피.음료 | 패스트푸드점 | 치킨전문점 | 슈퍼마켓 | 
한식음식점

📊 강남구의 한식음식점 업종 추천지역 Top10 결과 📊


,gu_name,dong_name,total_star
275,강남구,역삼1동,76
9,강남구,삼성1동,70
188,강남구,논현2동,67
88,강남구,압구정동,64
267,강남구,청담동,54
300,강남구,대치2동,54
42,강남구,논현1동,52
158,강남구,대치4동,48
390,강남구,신사동,41
156,강남구,도곡2동,39
